# **ASSIGNMENT 1: EMPIRICAL STUDY OF KNAPSACK PROBLEM**

This was an assignment completed for the Introduction to AI course. This assingnment was completed in a group of two people.
Portions of this assignment were removed to make it public. This is not a working/full copy of the completed assignment.

**2. Knapsack Problem**

The problem involves a bag and several items. The bag has a max weight W while the items are indexed, denoted Xi (0 if present, 1 otherwise), each with weight Wi and price Pi. The goal is to maximize the total price while keeping the total weight at or below W; that is, throughout all possible combinations, ∑XiPi is maximal and ∑XiWi ≤ W.



**3. Dataset**

We use a public domain dataset from Kaggle: https://www.kaggle.com/datasets/warcoder/knapsack-problem?resource=download
<br>
The dataset provided is a comma-separated-value file (.csv) with 5 columns:
1.   Weights: a 1x5 row vector denoting the weight for the item at the ith position.
2.   Prices: a 1x5 row vector denoting the price for the item at the ith position.
3.   Capacity: an integer denoting the bag's max capacity.
4.   Best picks: a 1x5 row vector with entries in {0, 1} denoting the presence (or absence) of an item.
5.   Best price: the total price resulting from best picks, ∑XiPi



**Import important libraries**

In [ ]:
import pandas as pd
import itertools
import numpy as np

**Read Dataset**

The link to the raw version is as follows:
*https://raw.githubusercontent.com/GITHUB_USERNAME/REPOSITORY_NAME/main/DATASETNAME.csv*

Now provide the link to YOUR dataset and read the dataset using pandas:

In [ ]:
#url removed for public copy
#url = "https://raw.githubusercontent.com/{knapsack_5_items.csv}"

dataset = pd.read_csv(url)

Let's see what are the columns of the dataset? :

In [ ]:
dataset.columns

Index(['Weights', 'Prices', 'Capacity', 'Best picks', 'Best price'], dtype='object')

As we expected, we have columns for weights, costs, capacity, best picks and best price for all the instances.

Now let's see the first 10 entries (rows):

In [ ]:
dataset.head(10)

,Weights,Prices,Capacity,Best picks,Best price
0,[46 40 42 38 10],[12 19 19 15 8],40,[0. 1. 0. 0. 0.],19.0
1,[11 31 4 6 7],[ 2 8 18 16 3],64,[1. 1. 1. 1. 1.],47.0
2,[32 49 27 37 24],[19 16 16 4 1],87,[1. 0. 1. 0. 1.],36.0
3,[20 35 22 23 16],[19 17 19 9 1],21,[1. 0. 0. 0. 0.],19.0
4,[ 7 12 19 13 20],[10 11 18 15 5],50,[0. 1. 1. 1. 0.],44.0
5,[27 10 25 25 7],[13 19 7 16 3],66,[1. 1. 0. 1. 0.],48.0
6,[21 2 33 45 26],[ 1 14 10 6 13],80,[0. 1. 1. 0. 1.],37.0
7,[37 27 39 14 25],[18 7 15 4 13],35,[0. 0. 0. 0. 1.],13.0
8,[ 1 48 4 23 39],[ 9 4 10 16 12],51,[1. 0. 1. 1. 0.],35.0
9,[ 4 3 22 9 32],[14 6 3 17 8],53,[1. 1. 0. 1. 1.],45.0


**Preprocessing Step**

Typically, the initial step in any project that involves reading and handling data is data preprocessing and cleansing.

In our dataset, we expect the entries in the "Weights," "Prices," and "Best Picks" columns to be in the form of arrays of floats or integers, like this: [45, 40, 42, 38, 10]

However, when you read each entry using pandas, they will be in a form of a string: "[45 40 42 38 10]"

So we need to convert these strings into "arrays of floats or integers." You can utilize the function provided below for this purpose:


In [ ]:
def string_to_list(string):

  string_list = string.strip('[]').split()

  float_list = [float(element) for element in string_list]

  return float_list

Furthermore, it's possible that certain rows in the dataset contain empty values in specific columns. We also aim to eliminate these rows as they do not provide any useful information. We use dropna() function to do so:

In [ ]:
#Ignore the warning messages.

dataset = dataset.dropna()

dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
dataset['Best picks'] = dataset['Best picks'].apply(lambda x : string_to_list(x))

Now it's time to implement the search algorithms. For each algorithm, a template is provided to you. You can modify this template if you want. But first you should try to go look at all the parameters used, as they are all important. You can also define any number of auxiliary functions you want.


**4. Generate and Test**

We generate all possible combinations of 0s and 1s on our 1x5 row vector. The generation is as intuitive as counting in binary, as we have for the first few: 00000, 00001, 00010 and so on... up until 11111; with a running total of 2^5 = 32 possibilities.

In [ ]:
# Input: data (a row that includes arrays like Weights, Prices & Best Picks as well as numbers like Capacity & Best price)
def gen_and_test(data):

  # Will store the current max (if current price > this and weight <= capacity)
  best_solution_price = 0

  # Stores the indeces for the best solution
  best_solution = []

  # solution code removed for public repo

  return best_solution_price, best_solution


In [ ]:
solutions = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = gen_and_test(row)
    solutions.append(1 if target == solution else 0)

In [ ]:
# Accuracy
print('Accuracy of best prices found is', np.mean(solutions))

Accuracy of best prices found is 1.0


------------------------------------------------------------------------------------------------

**5. Greedy Search**

The greedy search algorithm works by choosing the highest value for the current state. It will keep choosing the highest value item at each next state until it cannot fit more items into the bag. The greedy strategy used for this implementation was to choose the highest price first out of all the items.

References:
https://www.geeksforgeeks.org/fractional-knapsack-problem/
Fall2023-CSI4106-CSP Lecture Slides

In [ ]:
# the Item object with information on each item that can go in the bag
class Item:
  def __init__(self, weight, price, index):
    self.weight = weight # weight of item
    self.price = price # price or value
    self.index = index # index of the item in the original list
    self.ratio = price / weight # ratio is the price divided by the weight

# the greedy algorithm
def greedy(data):
  # create a new list for the Item objects
  items_list = []
  # loop through each index of the Prices list (which is all items in the list).
  for i in range(0, len(data['Prices'])):
    # create the Item object using the data from the weights list, prices lists, and indexes
    items_list.append(Item(data['Weights'][i], data['Prices'][i], i))

  # sort the item list using the price, sort from high to low / descending order
  items_list.sort(key=lambda x: x.price, reverse=True)

  # create a list of 0's with the length being the same as the Prices list
  # the list will be the length of the total number of objects to choose
  best_solution = [0] * len(data['Prices'])
  # start with best price of 0 and no items in the bag
  best_solution_price = 0
  # set the max capacity to the row data Capacity
  capacity = data['Capacity']
  # the current weight of the backpack starts at 0 with no items inside
  w = 0
  # the current total price or value in the backpack start at 0 with no items inside
  p = 0
  # loop through each item in the sorted list
  # the object with the highest price will be first in the list
  # and the lowest price is last in the list
  for item in items_list:
    # if the object's weight is added to the bag, and it is less than or equal to the bag capacity
    if (w + item.weight) <= capacity:
      # add the item's weight to the current weight in the bag
      w = w + item.weight
      # add the item's price to the total price in the bag
      p = p + item.price
      # put a 1 in the new best solution using the original list's index spot
      best_solution[item.index] = 1

  # the total price in the backpack is set to the current price that was found
  # after adding the items that could fit in the bag and be under capacity
  best_solution_price = p

  # return the best total price of bag solution and solution list for the items added or not
  return best_solution_price, best_solution


In [ ]:
solutions_greedy = []
recall = 0
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = greedy(row)
    solutions_greedy.append(1 if target == solution else 0)

    recall = recall + (1 if target == solution else 0)

In [ ]:
print("Greedy Accuracy is", np.mean(solutions_greedy))

In [ ]:
print("Recall is", recall, "optimal solutions and", len(solutions_greedy) - recall, "non optimal solutions.")

Recall is 8957 optimal solutions and 922 non optimal solutions.


------------------------------------------------------------------------------------------------

**6. Simulated Annealing**
The simulated annealing algorithm uses temperature to determine the amount of iterations by starting at a high temperature and decreases the temperature by a cooling rate. An initial solution is created using random or greedy, and while the temperature cools, a new solution is created at each iteration. This modified solution is similar to the intial solution and is accepted as the new solution to use in the next step based on a random value and a calculated probability based on the current temperature. This probability can make it so that worse solutions may be accepted. When the temperature reaches the min or the enrgy threshold is reached, then the final solution is found.

Reference: https://www.baeldung.com/cs/simulated-annealing (from lecture slides)
https://www.geeksforgeeks.org/simulated-annealing/


In [ ]:
from os import X_OK
import random
import math

# the function to maximize, the total price of items in the bag
def energy(data, sol):
  p = 0 # set total price to 0
  w = 0 # set total weight to 0
  for i in range(0, len(sol)): # for every item in the soln list
    if sol[i] == 1: # if the soln item is in the bag
      p = p + data['Prices'][i] # add the price of that item to p
      w = w + data['Weights'][i] # add the weight of that item to w
  # exceeding bag capacity is considered zero energy
  #if w > data['Capacity']:
   # p = 0
  # return the total price and weight of the items that are in the bag
  return p, w

# generate a new candidate solution by making a modification to the given soln
# random index to change from 0 to 1 or 1 to 0
def perturbation(sol):
  #copy the list to a new list
  new_sol = sol.copy()
  # get a random integer for the index to change in the list
  c = random.randint(0, len(sol)-1)
  # if the original list had a 0 in the random index c
  if sol[c] == 0:
    new_sol[c] = 1 # replace value 0 with 1
  else:
    # if the original list had a 1, then replace with 0
    new_sol[c] = 0
  return new_sol

# acceptance function will decide to keep the solution based on the energy
def accept(diff_e, t):
  # the candidate soln with the energy that is higher will always be accepted
  if (diff_e > 0): # if the difference of e is positive
    return True #accept the soln
  else: # the difference of e is negative
    r = random.random() # returns a random number in the range [0,1)
    # calculate the probability of acceptance for maximization
    if (r < math.exp(diff_e / t)): # modify the calculation to work with negative diff_e
      return True # accept the solution
    else:
      return False # reject the solution

# generate a random intial solution
def random_soln(data):
  sol = [] # start with an empty list
  p = 0 # start with price as 0
  # for each price of the items
  for i in data['Prices']:
    # choose either 0 or 1 to put the item into the bag or not
    r = random.randint(0, 1)
    # add the 0 or 1 value to the list
    sol.append(r)
    # if the item was added to the bag with 1
    if(r == 1):
      # add the price to the total initial price
      p = p + i
  #return total price and the items list for the bag
  return p, sol

# N is num of iterations
# refer to slide 22 or 24 of Fall2023-CSI4106-PopulationBased slides for the algorithm
def simulated_annealing(data, N, initial_temperature, cooling_rate):
  t_max = 100 # set the maximum temperature
  t_min = 1 # set the minimum temperature
  r = cooling_rate # cooling rate
  e_th = data['Best price'] * 0.80 # energy threshold is the 80 percent of the best price

  t = t_max # set the current temperature to the a high max temp

  # generate the initial candidate solution using greedy algorithm
  initial_price, x = greedy(data)
  #initial_price, x = random_soln(data)
  best_solution_price = initial_price # set current best price to inital soln price
  best_solution = x # set current best soln to intial soln

  # compute the energy of the intial soln
  e, w = energy(data, x)

  # while the temperature has not reached the minimum temp
  # and the energy has is still smaller than the threshold it needs to reach
  while (t > t_min) and (e < e_th):
    x_new = perturbation(x) # generate a new candidate soln
    e_new, w_new = energy(data, x_new) # compute the energy of the new candidate soln using the energy function
    diff_e = e_new - e # difference of the new soln energy and previous soln energy
    if accept(diff_e, t): # acceptance function
      # accept the new solution that was generated
      x = x_new # set the current soln to the new soln
      e = e_new # set the current energy to the new soln's energy
      w = w_new # set the current weight to the new soln's weight
      if (w <= data['Capacity']): # if the soln total weight is under the bag capacity
        best_solution = x # set a sthe new best soln
        best_solution_price = e # set as the new best total price
    t = r * t # cool/lower the temperature using the cooling rate

  # return the best final solution price and best soln list of items
  return best_solution_price, best_solution

In [ ]:
solutions_sa = []
recall_sa = 0
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 10, initial_temperature=1, cooling_rate=0.95)
    solutions_sa.append(1 if target == solution else 0)

    recall_sa = recall_sa + (1 if target == solution else 0)


In [ ]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))

In [ ]:
print("Recall is", recall_sa, "optimal solutions and", len(solutions_sa) - recall_sa, "non optimal solutions.")

Recall is 1506 optimal solutions and 8373 non optimal solutions.


------------------------------------------------------------------------------------------------

**7. Genetic Algorithm**

The genetic algorithm aims to draw parallels with the biological processes as purported by Charles Darwin. We have X(t) the population during the t-th cycle, and while in t, we aim to run tournaments to cull the weak (according to a fitness criteria; in this case, the total price), then crossbreed for the remaining champions and finally mutate them all, including the children. We end in the new population X(t+1) where we yet again repeat the process for num_generation times.

In [ ]:
import random

# Fitness in this case (Knapsack) will be the total price if weight <= capacity
def calculate_fitness(ind, prices, weights, capacity):
  fitness = 0

  # removed solution for public repo

  return fitness

def crossover(parent1, parent2, cross_rate):

  child1 = parent1
  child2 = parent2
  # removed solution for public repo

  return child1, child2

def mutation(child, mut_rate):

  # removed solution for public repo

  return child

def genetic_algorithm(data, population_size, num_generations, mut_rate, cross_rate, tournament_size):
  best_solution_price = 0
  best_solution = 0

  # removed solution for public repo

  return best_solution_price, best_solution

In [ ]:
solutions_ga = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = genetic_algorithm(row, population_size = 24, num_generations = 1, mut_rate = 0.1, cross_rate = 0.1, tournament_size = 2)
    solutions_ga.append(1 if target == solution else 0)


In [ ]:
print("Genetic Algorithm Accuracy is", np.mean(solutions_ga))

  **Your Analysis:**


*   Runtime Complexity: the implementation is very slow particularly because of the sheer amount of data that is being handled. On top of that, depending on how large population_size, num_generations and tournament_size are, the total runtime can increase as it increases the loops and subsequently the operations, like crossing and mutating which both require calls to np.random.
*   Optimal Solution: given high mut_rate & cross_rate, the solution can vary widely, but on average performs poorly. This is due to the optimality in the underlying 2 algorithms, Generate & Test plus Greedy. They both yield >80% accuracy always (since they are deterministic). Thus, for the best solutions we should attempt to shoot for lower rates of crossing and mutation since our solutions are already so optimized.
*   Practicality: given the small size of 5 for each Knapsack problem, the practicality is arguable since most configurations yield suboptimal accuracies; much lower than running them directly from the Generate & Test or Greedy algorithms. This does mean that for larger sizes, there can be more to be exploited from the Genetic Algorithm

------------------------------------------------------------------------------------------------

**8. Comparative Study**






In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data = [["Generate & Test", 100, 7], ["Greedy", 90.67, 1], ["Simulated Annealing", 90.73, 2], ["Generate & Test", 90.68, 17]]
pd.DataFrame(data, columns=["Algorithm", "Accuracy (%)", "Runtime (seconds)"])

,Algorithm,Accuracy (%),Runtime (seconds)
0,Generate & Test,100.00,7
1,Greedy,90.67,1
2,Simulated Annealing,90.73,2
3,Generate & Test,90.68,17


Reference for graphs: https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook

**9. Conclusion**

Comment on the empirical study, its results, and give ideas for future work.
Result Discussion:
1.    Generate & Test: the algorithm always provides the optimal solution given that it explores all branches/possibilities at a disadvantaged runtime. It is impractical to run the algorithm for even larger datasets and/or no. of objects, given that with our implementation we see a runtime of about 6 seconds.
2. Greedy: the algorithm provides 90% accuracy for optimal solutions. The greedy alg takes the best price at each state until it cannot fit any more items in the bag. Changing the greedy strategy can effect the accuracy to make it lower.
3. Simulated Annealing: the accuracy for simuated annealing depends on several factors such as temperature (min and max), cooling rate, energy threshold, and inital solution generation. Greedy increases the accuracy comapred to a random intial solution. The more iterations that the alg goes through can lead to higher accuracy. This algroithm can takes worse solutions based on a calculated probability as the temperature decreases. This randomness and modifying the solution slighlty can lead to more optimal solutions. THerefore the accuracy was higher than the greedy solution.
4.    Genetic Algorithm: from our comparative study, it goes to show that randomness can produce optimal results given the right parameters. That is, if we see that our greedy produces relatively high accuracy results, we should opt for lower mutation and/or crossover rates. Thus, we recommend running the genetical algorithm if we see that our greedy/random algorithm gets stuck pretty often. In other words, random implementations can be optimized by running them through the genetic algorithm since it incurs an element of randomness; i.e.: our point in the solution space can be brought closer to the global max.


--------------------------------------------------------------------------


**10. References**

*   Lecture Notes (From Week 3 and below)
*   https://stackoverflow.com/questions/3121979/how-to-sort-a-list-tuple-of-lists-tuples-by-the-element-at-a-given-index
*   https://www.geeksforgeeks.org/genetic-algorithms/

* Greedy:
* https://www.w3schools.com/python/pandas/ref_df_iterrows.asp
* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html
* https://stackoverflow.com/questions/522563/accessing-the-index-in-for-loops
* https://www.geeksforgeeks.org/greedy-algorithms-general-structure-and-applications/
* https://www.geeksforgeeks.org/fractional-knapsack-problem/
* https://www.geeksforgeeks.org/greedy-algorithms-general-structure-and-applications/
* https://www.geeksforgeeks.org/fractional-knapsack-problem/
* https://stackoverflow.com/questions/522563/accessing-the-index-in-for-loops
* https://www.geeksforgeeks.org/break-continue-and-pass-in-python/
* Simulated Annealing
* https://docs.python.org/3/library/random.html
* https://stackoverflow.com/questions/6088077/how-to-get-a-random-number-between-a-float-range
* https://stackoverflow.com/questions/30030659/in-python-what-is-the-difference-between-random-uniform-and-random-random
* https://www.w3schools.com/python/ref_random_random.asp
* https://stackoverflow.com/questions/33359740/random-number-between-0-and-1
* https://www.w3schools.com/python/ref_math_exp.asp
* https://www.w3schools.com/python/ref_func_range.asp
* https://www.w3schools.com/python/ref_random_randint.asp
* https://www.w3schools.com/python/python_lists_copy.asp
* https://www.geeksforgeeks.org/sum-function-python/
* https://www.geeksforgeeks.org/simulated-annealing/
* https://www.w3schools.com/python/ref_math_exp.asp
* https://www.geeksforgeeks.org/python-random-function/
* https://www.w3schools.com/python/ref_func_range.asp
* https://visualstudiomagazine.com/articles/2021/12/01/traveling-salesman.aspx

* Comparative study
* https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook

